In [ ]:
import pandas as pd
import numpy as np
from joblib import dump

import keras
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
keras.backend.set_floatx('float64')

from sklearn.preprocessing import StandardScaler

In [ ]:
params = ['alpha', 'rho', 'v', 'T', 'K']
DATA_PATH = r'C:\Users\hugo\OneDrive\Documents\Quantitative Finance\Thesis\Data\training_data'

### Import training data

In [ ]:
training_data = pd.read_pickle(DATA_PATH + '/training_data_grid.pkl')
random_data = pd.read_pickle(DATA_PATH + '/random_data.pkl')
random_data = random_data.sample(frac=1).reset_index(drop=True)

In [ ]:
split_random = 2200000
test_data_length = 2000000
training_data = training_data.append(random_data[:split_random])
test_data = random_data[split_random:split_random + test_data_length]
heatmap_data = random_data[split_random + test_data_length:]

In [ ]:
training_data.to_pickle(DATA_PATH + '/training_data.pkl')

In [ ]:
test_data = pd.read_pickle(DATA_PATH + '/validation_data.pkl') # these are 2M simulations

In [ ]:
heatmap_data.to_pickle(DATA_PATH + '/heatmap_data.pkl')

### Create final data sets

In [ ]:
train = training_data
test = test_data

x_train = train[params].to_numpy()
y_train = train['impl_volatility'].to_numpy().reshape(-1, 1)

x_test = test[params].to_numpy()
y_test = test['impl_volatility'].to_numpy().reshape(-1, 1)

### Scale the data

In [ ]:
scale_x = StandardScaler()
scale_y = StandardScaler()

x_train_transform = scale_x.fit_transform(x_train)
x_test_transform = scale_x.transform(x_test)

y_train_transform = scale_y.fit_transform(y_train)
y_test_transform = scale_y.transform(y_test)

### Create Keras model

In [ ]:
input_params = keras.layers.Input(shape=(5,))
hidden_1 = keras.layers.Dense(30, activation='elu')(input_params)
hidden_2 = keras.layers.Dense(30, activation='elu')(hidden_1)
hidden_3 = keras.layers.Dense(30, activation='elu')(hidden_2)
hidden_4 = keras.layers.Dense(30, activation='elu')(hidden_3)

output_layer = keras.layers.Dense(1, activation='linear')(hidden_4)


model = keras.models.Model(inputs=input1, outputs=output_layer)
model.summary()

### Fit the model

In [ ]:
def mean_squared_error(y_true, y_pred):
        return K.mean(K.square(y_pred - y_true))

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)
model.compile(loss = mean_squared_error, optimizer = "adam")
model.fit(x_train_transform, y_train_transform, batch_size=2048, validation_data=(x_test_transform, y_test_transform),epochs = 200, verbose = True,shuffle=1,callbacks=[earlystop])
history = model.history.history

### Save model parameters

In [ ]:
save_path = r'C:\Users\hugo\OneDrive\Documents\Quantitative Finance\Thesis\Data\ann'
modelGEN.save_weights(save_path + '\weights_pointwise.h5')
dump(scale_y, save_path + '\scale_y_pointwise.bin', compress=True)
dump(scale_x, save_path + '\scale_x_pointwise.bin', compress=True)

loss = cost function training data

val_loss = cost function validation data

In [ ]:
# Save history
pd.DataFrame(history).to_pickle(r"C:\Users\hugo\OneDrive\Documents\Quantitative Finance\Thesis\Data\history_ANN.pkl")